In [1]:
import pandas as pd
import networkx as nx
from networkx.algorithms import bipartite
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import timedelta
import os

In [2]:
data_dir = 'C:/Users/udayk/OneDrive - stevens.edu/Documents/GitHub/sna-project/data'
os.chdir(data_dir)
filename='email-Eu-core-temporal.txt'
# Read the dataset
data = pd.read_csv(filename, delimiter=" ", header=None, names=["node1", "node2", "weight"])

In [3]:
# Create a graph
G = nx.Graph()

In [4]:
# Add edges with weight attribute
for idx, row in data.iterrows():
    G.add_edge(row["node1"], row["node2"], weight=row["weight"])

# Calculate some network metrics
num_nodes = G.number_of_nodes()
num_edges = G.number_of_edges()
density = nx.density(G)
degree_distribution = dict(G.degree())
average_degree = np.mean(list(degree_distribution.values()))

In [5]:
# Print metrics
print(f"Number of nodes: {num_nodes}")
print(f"Number of edges: {num_edges}")
print(f"Density: {density}")
print(f"Average degree: {average_degree}")

# Perform ERGM
# ERGM is currently not supported directly in NetworkX, but you can use the R package `statnet` with the `rpy2` package to call R functions from Python
# Installation: pip install rpy2
# In R: install.packages('statnet')

Number of nodes: 986
Number of edges: 16064
Density: 0.033080384262929745
Average degree: 32.5841784989858


In [6]:
import rpy2.robjects as ro
from rpy2.robjects.packages import importr
from rpy2.robjects import r, pandas2ri
from rpy2.robjects.conversion import localconverter

In [7]:
statnet = importr("statnet")
pandas2ri.activate()

# Convert the NetworkX graph to an R 'network' object
with localconverter(ro.default_converter + pandas2ri.converter):
    r_data = ro.conversion.py2rpy(data)

--- Please select a CRAN mirror for use in this session ---


In [9]:
print(r_data)

      node1 node2  weight
0       582   364       0
1       168   472    2797
2       168   912    3304
3         2   790    4523
4         2   322    7926
5         2   790    8061
6        42   402   19403
7       870   337   19560
8       663   362   21077
9       663   410   21280
10      565   550   21793
11      987   433   21799
12      663   343   22038
13      581   135   23175
14      581   942   23263
15      978   988   23553
16      978   421   23553
17      978   795   23553
18      978   547   23553
19      978   851   23553
20      978   469   23553
21      978   188   23553
22      978   614   23553
23      978   301   23553
24      978   954   23553
25      978   817   23553
26      978   167   23553
27      978   141   23553
28      978   552   23553
29      978   424   23553
30      978    35   23553
31      978   207   23553
32      569   749   23646
33      174   534   23652
34      844   684   23712
35       31   165   23813
36      429   401   24004
37      569 

In [12]:
r_edges = r("as.matrix")(r_data)


In [13]:
r_graph = statnet.network(r_edges, matrix_type="edgelist", directed=False)

AttributeError: module 'statnet' has no attribute 'network'

In [ ]:

# Perform ERGM
ergm_model = statnet.ergm(r_graph ~ statnet.edges() + statnet.nodematch("weight"))
print(ergm_model)

In [10]:
# Perform ERGM
ergm_formula = "edges | nodematch('weight')"
ergm_model = statnet.ergm(r_graph ~ ergm_formula)
print(ergm_model)


SyntaxError: invalid syntax (1349201938.py, line 3)